In [4]:
import pandas as pd
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.github.hooks.github import GithubHook
import datetime
def load_to_github():
    # Cargar el archivo CSV en un dataframe de pandas
    df = pd.read_csv("https://raw.githubusercontent.com/Lina-Arroyo/PruebaTecnica/main/Data/Traffic_Flow_Map_Volumes.csv")

    # Conectar con GitHub
    hook = GithubHook(github_conn_id="ConecctionGit")
    repo = hook.get_repo("Lina-Arroyo/PruebaTecnica")

    # Escribir el dataframe en el repositorio de GitHub
    file = repo.get_contents("Data-transformed/Traffic_Flow_Map_Volumes_TRANSFORMED.csv")
    repo.update_file(file.path, "commit message", df.to_csv(index=False), file.sha)

In [5]:
load_to_github()

AirflowNotFoundException: The conn_id `ConecctionGit` isn't defined

In [ ]:
import pandas as pd
from airflow.operators.bash import BashOperator
 
def save_dataframe_to_csv():
    # Load dataframe from source
    df = pd.read_csv("https://raw.githubusercontent.com/Lina-Arroyo/PruebaTecnica/main/Data/Traffic_Flow_Map_Volumes.csv")


save_dataframe_to_csv_operator = PythonOperator(
    task_id='save_dataframe_to_csv',
    python_callable=save_dataframe_to_csv,
    dag=dag
)

add_to_github_operator = BashOperator(
    task_id='add_to_github',
    bash_command='cd PruebaTecnica/Data-transformed/ && git add Traffic_Flow_Map_Volumes_TRANSFORMED.csv && git commit -m "Data transformada subida con exito" && git push',
    dag=dag
)

save_dataframe_to_csv_operator >> add_to_github_operator
